In [1]:
# import the necessary packages
import os
import cv2
import argparse
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [2]:
# set the paths on where the prototxt, caffemodel and model files are 
# and other parameters
prototxt_path = "/Users/Tejaswini/Documents/CV/Facemask Detection/face/deploy.prototxt"
weights_path = "/Users/Tejaswini/Documents/CV/Facemask Detection/face/res10_300x300_ssd_iter_140000.caffemodel"
model_path = "/Users/Tejaswini/Documents/CV/Facemask Detection/model"
input_image_path = "/Users/Tejaswini/Documents/CV/Facemask Detection/examples/example2.jpeg"
minimum_confidence = 0.5

In [3]:
# load our serialized face detector model from disk
print("Loading face detector model...")
net = cv2.dnn.readNet(prototxt_path, weights_path)

Loading face detector model...


error: OpenCV(4.5.5) /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/caffe/caffe_io.cpp:1126: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "/Users/Tejaswini/Documents/CV/Facemask Detection/face/deploy.prototxt" in function 'ReadProtoFromTextFile'


In [ ]:
# load the face mask detector model from disk
print("Loading face mask detector model...")
model = load_model(model_path)

In [ ]:
# load the input image from disk, clone it, and grab the image spatial
# dimensions
input_image = cv2.imread(input_image_path)
orig = input_image.copy()
(h, w) = input_image.shape[:2]

In [ ]:
# construct a blob from the image
blob = cv2.dnn.blobFromImage(input_image, 1.0, (300, 300), (104.0, 177.0, 123.0))

In [ ]:
# pass the blob through the network and obtain the face detections
print("Computing face detections...")
net.setInput(blob)
detections = net.forward()

In [ ]:
# loop over the detections
for index in range(0, detections.shape[2]):
    # extract the confidence (i.e., probability) associated with
    # the detection
    confidence = detections[0, 0, index, 2]

    # filter out weak detections by ensuring the confidence is
    # greater than the minimum confidence
    if confidence > minimum_confidence:
        # compute the (x, y)-coordinates of the bounding box for
        # the object
        box = detections[0, 0, index, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        # ensure the bounding boxes fall within the dimensions of
        # the frame
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

        # extract the face ROI, convert it from BGR to RGB channel
        # ordering, resize it to 224x224, and preprocess it
        face = input_image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        # pass the face through the model to determine if the face
        # has a mask or not
        (mask, withoutMask) = model.predict(face)[0]
        
        # determine the class label and color we'll use to draw
        # the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(input_image, label, (startX, startY - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(input_image, (startX, startY), (endX, endY), color, 2)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(input_image)